In [211]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython import display
import pymongo
import random
import numpy as np
from bokeh.io import output_notebook
from climatools.cliradlw.cliradlw_latest import *
from climatools.html.html import *
from climatools.spectrum import *
output_notebook()
pd.set_option('max_colwidth', -1)

Loading BokehJS ...

In [237]:
LINKS.fillna('', inplace=True)
LINKS.index.names = ['absorber(s)']
LINKS.sort_index(axis=1, level=1, inplace=True)
LINKS.sort_index(axis=0, inplace=True)
display.display(display.HTML(LINKS.to_html(escape=False)))

lblnew band(s),[9],[3a],[3b],[3c],[4],[5],[7]
cliradlw band(s),[11],[3],[4],[5],[6],[7],[9]
absorber(s),,,,,,,
"{'ch4': 1.8e-06, 'h2o': 'atmpro', 'n2o': 6.4e-07}",,,,,,,mls saw trp
"{'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 6.4e-07}",,mls saw trp,,,,,
{'co2': 0.0008},mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,
"{'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07}",,mls saw trp,,,,,
"{'h2o': 'atmpro', 'co2': 0.0008, 'o3': 'atmpro'}",,,,,,mls saw trp,
"{'h2o': 'atmpro', 'co2': 0.0008}",mls saw trp,,mls saw trp,mls saw trp,mls saw trp,,
{'n2o': 6.4e-07},,mls saw trp,,,,,mls saw trp


In [185]:
CLIENT = pymongo.MongoClient(host='localhost', port=27017)

In [203]:
LINKS = pd.DataFrame(columns=pd.MultiIndex(levels=[[], []], labels=[[], []], names=['lblnew band(s)', 'cliradlw band(s)']))
LINKS.index.name = ['absorber(s)']

# 2 x $CO_2$

In [204]:
MOL = 'co2'
CONC = 2 * 400e-6
COMMITNUMBER = '1013f91'
NV, DV = 1000, .001
ATMPRO = dict(mls=294, saw=257, trp=300)

### Atmosphere compositions

We first set the concentration of CO2 to that double of the usual 4e-6 ppmv.  Then we split the overall composition into those where CO2 overlaps with other gases and those where only CO2 is present.

In [6]:
atmcomp = AtmComposition.cliradlw_nongreys()
atmcomp.gasconcs['co2'] = CONC
atmcomp.gasinbands = {b:gs for b, gs in atmcomp.gasinbands.items() if MOL in gs}

In [7]:
atmcomps_olp = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b: gs}) 
                for b, gs in atmcomp.gasinbands.items() if len(gs) > 1]

In [8]:
atmcomps_bft = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b:[MOL]}) for b, _ in atmcomp.gasinbands.items()]

### $CO_2$-only 

In [207]:
for atmbft in atmcomps_bft:
    for atmpro in ATMPRO.keys():
        parm_cli = atmbft.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])
        data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)
    
        parms_lbl = atmbft.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], nv=NV, dv=DV)
        parm_lbl = parms_lbl[0]
        parm_lbl.avail_commits(collection=CLIENT.lblnew.bestfit_lw)
        data_lbl = parm_lbl.modeldata_pymongo(collection=CLIENT.lblnew.bestfit_lw)
    
        ida = show_results(dlbl=data_lbl, dcli=data_cli, add_idanchor=True)
        remove_href(df=LINKS, idanchor=ida, dlbl=data_lbl, dcli=data_cli)
        show_href(df=LINKS, idanchor=ida, dlbl=data_lbl, dcli=data_cli)

-----------------------------

<a id="55165445240"></a>

[3] {'co2': 0.0008} mls

Run parameters  \
                                CLIRAD   
atmpro                 mls               
band                   [3]               
commitnumber           1013f91           
conc                   -                 
dv                     -                 
klin                   -                 
molecule               {'co2': 0.0008}   
ng_adju                -                 
ng_refs                -                 
nv                     -                 
option_compute_btable  -                 
option_compute_ktable  -                 
option_wgt_flux        -                 
option_wgt_k           -                 
ref_pts                -                 
tsfc                   294               
vmax                   -                 
vmin                   -                 
w_diffuse              -                 
wgt                    -                 

                                                                                 \
                                                                         LBLNEW   
atmpro                 mls                                                        
band                   3a                                                         
commitnumber           a22ab94                                                    
conc                   0.0008                                                     
dv                     0.001                                                      
klin                   6.37556e-24                                                
molecule               co2                                                        
ng_adju                [0, 0, 0]                                                  
ng_refs                [3, 2, 4]                                                  
nv                     1000                                                       
option_compute_btable  0                                                          
option_compute_ktable  0                                                          
option_wgt_flux        1                                                          
option_wgt_k           1                                                          
ref_pts                [[1, 250], [10, 250], [500, 250]]                          
tsfc                   294                                                        
vmax                   620                                                        
vmin                   540                                                        
w_diffuse              [[1.6, 1.6, 1.7], [1.75, 1.75], [1.55, 1.55, 1.6, 1.85]]   
wgt                    [[0.7, 0.3, 0.7], [0.7, 0.6], [0.4, 0.5, 0.8, 0.95]]       

                                                                 Fit parameters  
                                                                      (co2, 3a)  
atmpro                 mls                                                       
band                   3a                                                        
commitnumber           a06b618                                                   
conc                   0.0004                                                    
dv                     0.001                                                     
klin                   6.37556e-24                                               
molecule               co2                                                       
ng_adju                [0, 0, 0]                                                 
ng_refs                [3, 2, 4]                                                 
nv                     1000                                                      
option_compute_btable  0                                                         
option_compute_ktable  1                                                         
option_wgt_flux        1                                                         
option_wgt_k           1                  

--------------------------

...

<a id="77280085687"></a>

[11] {'co2': 0.0008} trp

Run parameters  \
                                CLIRAD   
atmpro                 trp               
band                   [11]              
commitnumber           1013f91           
conc                   -                 
dv                     -                 
klin                   -                 
molecule               {'co2': 0.0008}   
ng_adju                -                 
ng_refs                -                 
nv                     -                 
option_compute_btable  -                 
option_compute_ktable  -                 
option_wgt_flux        -                 
option_wgt_k           -                 
ref_pts                -                 
tsfc                   300               
vmax                   -                 
vmin                   -                 
w_diffuse              -                 
wgt                    -                 

                                                                \
                                                        LBLNEW   
atmpro                 trp                                       
band                   9                                         
commitnumber           a22ab94                                   
conc                   0.0008                                    
dv                     0.001                                     
klin                   6.5e-24                                   
molecule               co2                                       
ng_adju                [0, 0]                                    
ng_refs                [3, 3]                                    
nv                     1000                                      
option_compute_btable  0                                         
option_compute_ktable  0                                         
option_wgt_flux        1                                         
option_wgt_k           1                                         
ref_pts                [[1, 250], [50, 250]]                     
tsfc                   300                                       
vmax                   3000                                      
vmin                   1900                                      
w_diffuse              [[1.66, 1.66, 1.75], [1.75, 1.6, 1.85]]   
wgt                    [[0.7, 0.8, 0.7], [0.8, 0.7, 0.8]]        

                                                Fit parameters  
                                                      (co2, 9)  
atmpro                 mls                                      
band                   9                                        
commitnumber           a06b618                                  
conc                   0.0004                                   
dv                     0.001                                    
klin                   6.5e-24                                  
molecule               co2                                      
ng_adju                [0, 0]                                   
ng_refs                [3, 3]                                   
nv                     1000                                     
option_compute_btable  0                                        
option_compute_ktable  1                                        
option_wgt_flux        1                                        
option_wgt_k           1                                        
ref_pts                [(1, 250), (50, 250)]                    
tsfc                   294                                      
vmax                   3000                                     
vmin                   1900                                     
w_diffuse              [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]  
wgt                    [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]

--------------------------

<a href="#86532176100">trp</a> <a href="#58688352127">mls</a> <a href="#07403047707">saw</a>
<a href="#58688352127">mls</a> <a href="#07403047707">saw</a>


### $CO_2$ overlapped with other gases

In [212]:
for atmolp in atmcomps_olp:
    for atmpro in ATMPRO.keys():
        parm_cli = atmolp.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])
        data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)
        
        parms_lbl = atmolp.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], dv=DV, nv=NV)
        parm_lbl = parms_lbl[0]
        parm_lbl.avail_commits(collection=CLIENT.lblnew.overlap_lw)
        data_lbl = parm_lbl.modeldata_pymongo(collection=CLIENT.lblnew.overlap_lw)
        
        ida = show_results(dlbl=data_lbl, dcli=data_cli, add_idanchor=True)
        remove_href(df=LINKS, idanchor=ida, dlbl=data_lbl, dcli=data_cli)
        show_href(df=LINKS, idanchor=ida, dlbl=data_lbl, dcli=data_cli)

-----------------------------

<a id="64267022481"></a>

[3] {'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07} mls

Run parameters  \
                                                                 CLIRAD   
atmpro                 mls                                                
band                   [3]                                                
commitnumber           1013f91                                            
conc                   -                                                  
dv                     -                                                  
klin                   -                                                  
molecule               {'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07}   
ng_adju                -                                                  
ng_refs                -                                                  
nv                     -                                                  
option_compute_btable  -                                                  
option_compute_ktable  -                                                  
option_wgt_flux        -                                                  
option_wgt_k           -                                                  
ref_pts                -                                                  
tsfc                   294                                                
vmax                   -                                                  
vmin                   -                                                  
w_diffuse              -                                                  
wgt                    -                                                  

                                                                         \
                                                                 LBLNEW   
atmpro                 mls                                                
band                   3a                                                 
commitnumber           a22ab94                                            
conc                   -                                                  
dv                     0.001                                              
klin                   -                                                  
molecule               {'co2': 0.0008, 'h2o': 'atmpro', 'n2o': 3.2e-07}   
ng_adju                -                                                  
ng_refs                -                                                  
nv                     1000                                               
option_compute_btable  -                                                  
option_compute_ktable  -                                                  
option_wgt_flux        -                                                  
option_wgt_k           -                                                  
ref_pts                -                                                  
tsfc                   294                                                
vmax                   -                                                  
vmin                   -                                                  
w_diffuse              -                                                  
wgt                    -                                                  

                                                                 Fit parameters  \
                                                                      (co2, 3a)   
atmpro                 mls                                                        
band                   3a                                                         
commitnumber           a06b618                                                    
conc                   0.0004                                                     
dv                     0.001                                                      
klin                   6.37556e-24                                                
molecule               co2                                                        
ng_adju 

--------------------------

...

<a id="18240277413"></a>

[11] {'h2o': 'atmpro', 'co2': 0.0008} trp

Run parameters  \
                                                 CLIRAD   
atmpro                 trp                                
band                   [11]                               
commitnumber           1013f91                            
conc                   -                                  
dv                     -                                  
klin                   -                                  
molecule               {'h2o': 'atmpro', 'co2': 0.0008}   
ng_adju                -                                  
ng_refs                -                                  
nv                     -                                  
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   300                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                         \
                                                 LBLNEW   
atmpro                 trp                                
band                   9                                  
commitnumber           a22ab94                            
conc                   -                                  
dv                     0.001                              
klin                   -                                  
molecule               {'co2': 0.0008, 'h2o': 'atmpro'}   
ng_adju                -                                  
ng_refs                -                                  
nv                     1000                               
option_compute_btable  -                                  
option_compute_ktable  -                                  
option_wgt_flux        -                                  
option_wgt_k           -                                  
ref_pts                -                                  
tsfc                   300                                
vmax                   -                                  
vmin                   -                                  
w_diffuse              -                                  
wgt                    -                                  

                                                Fit parameters  \
                                                      (co2, 9)   
atmpro                 mls                                       
band                   9                                         
commitnumber           a06b618                                   
conc                   0.0004                                    
dv                     0.001                                     
klin                   6.5e-24                                   
molecule               co2                                       
ng_adju                [0, 0]                                    
ng_refs                [3, 3]                                    
nv                     1000                                      
option_compute_btable  0                                         
option_compute_ktable  1                                         
option_wgt_flux        1                                         
option_wgt_k           1                                         
ref_pts                [(1, 250), (50, 250)]                     
tsfc                   294                                       
vmax                   3000                                      
vmin                   1900                                      
w_diffuse              [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]   
wgt                    [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]

--------------------------

<a href="#51472567644">mls</a> <a href="#57004222267">saw</a> <class 'str'>
<a href="#51472567644">mls</a> <a href="#57004222267">saw</a>


# 2 x $N_2O$ 

In [214]:
MOL = 'n2o'
CONC = 2 * 3.2e-7
COMMITNUMBER = ''
NV, DV = 1000, .001

### Atmospere composition

In [221]:
atmcomp = AtmComposition.cliradlw_nongreys()
atmcomp.gasconcs['n2o'] *= 2

atmcomps_bft = [AtmComposition(gasinbands={band:[MOL]}, gasconcs=atmcomp.gasconcs) 
                for band, gs in atmcomp.gasinbands.items() if MOL in gs]
atmcomps_olp = [AtmComposition(gasinbands={band:gs}, gasconcs=atmcomp.gasconcs)
                for band, gs in atmcomp.gasinbands.items() if MOL in gs and len(gs) > 1]

### $N_2O$-only

In [229]:
for c in atmcomps_bft:
    for atmpro in ATMPRO.keys():
        pcli = c.to_cliradparam(atmpro=atmpro, tsfc=ATMPRO[atmpro])
        #print(pcli.avail_commits(collection=CLIENT.cliradnew.lw))
        dcli = pcli.modeldata_pymongo(collection=CLIENT.cliradnew.lw, commits=['523cbb7'])
        
        plbls= c.to_lblnewparam(bestfit_values=True, nv=NV, dv=DV, atmpro=atmpro, tsfc=ATMPRO[atmpro])
        plbl = plbls[0]
        #print(plbl.avail_commits(collection=CLIENT.lblnew.bestfit_lw))
        dlbl = plbl.modeldata_pymongo(collection=CLIENT.lblnew.bestfit_lw, commits=['a22ab94'])
        
        ida = show_results(dlbl=dlbl, dcli=dcli, add_idanchor=True)
        remove_href(idanchor=ida, df=LINKS, dlbl=dlbl, dcli=dcli)
        show_href(idanchor=ida, df=LINKS, dlbl=dlbl, dcli=dcli)
        
        

-----------------------------

<a id="32703143467"></a>

[3] {'n2o': 6.4e-07} mls

Run parameters                          \
                                 CLIRAD                  LBLNEW   
atmpro                 mls               mls                      
band                   [3]               3a                       
commitnumber           523cbb7           a22ab94                  
conc                   -                 6.4e-07                  
dv                     -                 0.001                    
klin                   -                 2.22e-20                 
molecule               {'n2o': 6.4e-07}  n2o                      
ng_adju                -                 [0, 0]                   
ng_refs                -                 [1, 2]                   
nv                     -                 1000                     
option_compute_btable  -                 0                        
option_compute_ktable  -                 0                        
option_wgt_flux        -                 1                        
option_wgt_k           -                 1                        
ref_pts                -                 [[1, 250], [500, 250]]   
tsfc                   294               294                      
vmax                   -                 620                      
vmin                   -                 540                      
w_diffuse              -                 [[1.8], [1.66, 1.8]]     
wgt                    -                 [[0.9], [0.5, 0.95]]     

                               Fit parameters  
                                    (n2o, 3a)  
atmpro                 mls                     
band                   3a                      
commitnumber           a06b618                 
conc                   3.2e-07                 
dv                     0.001                   
klin                   2.22e-20                
molecule               n2o                     
ng_adju                [0, 0]                  
ng_refs                [1, 2]                  
nv                     1000                    
option_compute_btable  0                       
option_compute_ktable  1                       
option_wgt_flux        1                       
option_wgt_k           1                       
ref_pts                [(1, 250), (500, 250)]  
tsfc                   294                     
vmax                   620                     
vmin                   540                     
w_diffuse              [(1.8,), (1.66, 1.8)]   
wgt                    [(0.9,), (0.5, 0.95)]

--------------------------

...

<a id="55688443226"></a>

[9] {'n2o': 6.4e-07} trp

Run parameters                             \
                                 CLIRAD                     LBLNEW   
atmpro                 trp               trp                         
band                   [9]               7                           
commitnumber           523cbb7           a22ab94                     
conc                   -                 6.4e-07                     
dv                     -                 0.001                       
klin                   -                 2.22e-20                    
molecule               {'n2o': 6.4e-07}  n2o                         
ng_adju                -                 [0, 0]                      
ng_refs                -                 [2, 2]                      
nv                     -                 1000                        
option_compute_btable  -                 0                           
option_compute_ktable  -                 0                           
option_wgt_flux        -                 1                           
option_wgt_k           -                 1                           
ref_pts                -                 [[1, 250], [500, 250]]      
tsfc                   300               300                         
vmax                   -                 1380                        
vmin                   -                 1215                        
w_diffuse              -                 [[1.8, 1.66], [1.6, 1.8]]   
wgt                    -                 [[0.6, 0.5], [0.6, 0.9]]    

                                  Fit parameters  
                                        (n2o, 7)  
atmpro                 mls                        
band                   7                          
commitnumber           a06b618                    
conc                   3.2e-07                    
dv                     0.001                      
klin                   2.22e-20                   
molecule               n2o                        
ng_adju                [0, 0]                     
ng_refs                [2, 2]                     
nv                     1000                       
option_compute_btable  0                          
option_compute_ktable  1                          
option_wgt_flux        1                          
option_wgt_k           1                          
ref_pts                [(1, 250), (500, 250)]     
tsfc                   294                        
vmax                   1380                       
vmin                   1215                       
w_diffuse              [(1.8, 1.66), (1.6, 1.8)]  
wgt                    [(0.6, 0.5), (0.6, 0.9)]

--------------------------

<a href="#03260377255">mls</a> <a href="#21154000387">saw</a> <class 'str'>
<a href="#03260377255">mls</a> <a href="#21154000387">saw</a>


In [220]:
atmcomp

<class 'climatools.atm.AtmComposition'>
Gases in each band:
{1: ['h2o'], 2: ['h2o'], 3: ['h2o', 'co2', 'n2o'], 4: ['h2o', 'co2'], 5: ['h2o', 'co2'], 6: ['h2o', 'co2'], 7: ['h2o', 'co2', 'o3'], 8: ['h2o'], 9: ['h2o', 'n2o', 'ch4'], 10: ['h2o'], 11: ['h2o', 'co2']}
Gas concentrations:
{'h2o': 'atmpro', 'co2': 0.0004, 'o3': 'atmpro', 'n2o': 6.4e-07, 'ch4': 1.8e-06}

### $N_2O$ overlapped with other gases

In [236]:
for c in atmcomps_olp:
    for atmpro in ATMPRO.keys():
        pcli = c.to_cliradparam(atmpro=atmpro, tsfc=ATMPRO[atmpro], commitnumber='523cbb7')
        #print('cliradlw', pcli.avail_commits(collection=CLIENT.cliradnew.lw))
        dcli = pcli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)
        
        plbls = c.to_lblnewparam(bestfit_values=True, nv=NV, dv=DV, atmpro=atmpro, tsfc=ATMPRO[atmpro], commitnumber='a22ab94')
        plbl = plbls[0]
        #print('lbl', plbl.avail_commits(collection=CLIENT.lblnew.overlap_lw))
        dlbl = plbl.modeldata_pymongo(collection=CLIENT.lblnew.overlap_lw)
        
        ida = show_results(dlbl=dlbl, dcli=dcli, add_idanchor=True)
        remove_href(idanchor=ida, df=LINKS, dlbl=dlbl, dcli=dcli)
        show_href(idanchor=ida, df=LINKS, dlbl=dlbl, dcli=dcli)

-----------------------------

<a id="68435851811"></a>

[3] {'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 6.4e-07} mls

Run parameters  \
                                                                 CLIRAD   
atmpro                 mls                                                
band                   [3]                                                
commitnumber           523cbb7                                            
conc                   -                                                  
dv                     -                                                  
klin                   -                                                  
molecule               {'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 6.4e-07}   
ng_adju                -                                                  
ng_refs                -                                                  
nv                     -                                                  
option_compute_btable  -                                                  
option_compute_ktable  -                                                  
option_wgt_flux        -                                                  
option_wgt_k           -                                                  
ref_pts                -                                                  
tsfc                   294                                                
vmax                   -                                                  
vmin                   -                                                  
w_diffuse              -                                                  
wgt                    -                                                  

                                                                         \
                                                                 LBLNEW   
atmpro                 mls                                                
band                   3a                                                 
commitnumber           a22ab94                                            
conc                   -                                                  
dv                     0.001                                              
klin                   -                                                  
molecule               {'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 6.4e-07}   
ng_adju                -                                                  
ng_refs                -                                                  
nv                     1000                                               
option_compute_btable  -                                                  
option_compute_ktable  -                                                  
option_wgt_flux        -                                                  
option_wgt_k           -                                                  
ref_pts                -                                                  
tsfc                   294                                                
vmax                   -                                                  
vmin                   -                                                  
w_diffuse              -                                                  
wgt                    -                                                  

                                                                 Fit parameters  \
                                                                      (co2, 3a)   
atmpro                 mls                                                        
band                   3a                                                         
commitnumber           a06b618                                                    
conc                   0.0004                                                     
dv                     0.001                                                      
klin                   6.37556e-24                                                
molecule               co2                                                        
ng_adju 

--------------------------

...

<a id="47175825262"></a>

[9] {'ch4': 1.8e-06, 'h2o': 'atmpro', 'n2o': 6.4e-07} trp

Run parameters  \
                                                                  CLIRAD   
atmpro                 trp                                                 
band                   [9]                                                 
commitnumber           523cbb7                                             
conc                   -                                                   
dv                     -                                                   
klin                   -                                                   
molecule               {'ch4': 1.8e-06, 'h2o': 'atmpro', 'n2o': 6.4e-07}   
ng_adju                -                                                   
ng_refs                -                                                   
nv                     -                                                   
option_compute_btable  -                                                   
option_compute_ktable  -                                                   
option_wgt_flux        -                                                   
option_wgt_k           -                                                   
ref_pts                -                                                   
tsfc                   300                                                 
vmax                   -                                                   
vmin                   -                                                   
w_diffuse              -                                                   
wgt                    -                                                   

                                                                          \
                                                                  LBLNEW   
atmpro                 trp                                                 
band                   7                                                   
commitnumber           a22ab94                                             
conc                   -                                                   
dv                     0.001                                               
klin                   -                                                   
molecule               {'ch4': 1.8e-06, 'h2o': 'atmpro', 'n2o': 6.4e-07}   
ng_adju                -                                                   
ng_refs                -                                                   
nv                     1000                                                
option_compute_btable  -                                                   
option_compute_ktable  -                                                   
option_wgt_flux        -                                                   
option_wgt_k           -                                                   
ref_pts                -                                                   
tsfc                   300                                                 
vmax                   -                                                   
vmin                   -                                                   
w_diffuse              -                                                   
wgt                    -                                                   

                                           Fit parameters  \
                                                 (ch4, 7)   
atmpro                 mls                                  
band                   7                                    
commitnumber           a06b618                              
conc                   1.8e-06                              
dv                     0.001                                
klin                   2e-21                                
molecule               ch4                                  
ng_adju                [0, 0]                               
ng_refs                [2, 3]                               
nv                     1000              

--------------------------

<a href="#83127347806">mls</a> <a href="#86126485022">saw</a> <class 'str'>
<a href="#83127347806">mls</a> <a href="#86126485022">saw</a>


# Fin

In [187]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
